<a href="https://colab.research.google.com/github/CeciliaMarson/CMLS-Homework1/blob/master/Andres_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5

In [0]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp
import pandas as pd

In [0]:
!wget --no-check-certificate -r "http://cvml.unige.ch/databases/DEAM/"

In [0]:
!unzip "cvml.unige.ch/databases/DEAM/DEAM_audio.zip"
!unzip "cvml.unige.ch/databases/DEAM/DEAM_Annotations.zip"
!unzip "cvml.unige.ch/databases/DEAM/features.zip"

# Get outputs and features from songs and compute the mean

Get output values train


In [0]:
cols = [' valence_mean',' arousal_mean'] # names of mean columns in annotations file
arousal = [' arousal_mean'] # name of arousal mean columns in annotations file
valence = [' valence_mean'] # name of valence mean columns in annotations file

# get dataframe from annotations file
annotations = pd.read_csv('/content/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv')

song_ids = annotations['song_id'] # get first column from dataframe
s_id = np.array(song_ids) # change first column from dataframe into numpy array


In [0]:
feats = os.listdir('features') # get list of all files inside features folder
feat_song_id = [s.strip('.csv') for s in feats] # get list of names (without file type) of files from features folder


In [0]:
# get numpy array of file names that are equal in both features folder and annotations file
same_song_id = np.intersect1d(s_id, feat_song_id)
same_song_id_l = same_song_id.tolist()
same_song_id_l.sort(key=int)
same_song_id_arr = np.array(same_song_id_l)
same_song = same_song_id_arr.reshape(len(same_song_id_arr),1)


In [0]:
# separate song ids randomly into a train set and a test set with a 80/20 ratio respectively
id_train, id_test = sklearn.model_selection.train_test_split(same_song_id_arr, test_size=0.2, random_state=42)
# id_train = 1395 #
# id_test = 349  #
id_train_l = id_train.tolist()
id_train_l.sort(key=int)
id_train_arr = np.array(id_train_l)

id_test_l = id_test.tolist()
id_test_l.sort(key=int)
id_test_arr = np.array(id_test_l)


In [0]:
audio_root = '/content/MEMD_audio/' # root path of audio files
audio_files = [n + '.mp3' for n in same_song_id_l] # audio files sorted from low to high

audio_path = []
for f in audio_files:
  audio_path.append(os.path.join(audio_root, f)) # audio files path from dataset

audio_path_samples = len(audio_path)


In [0]:
# function to compute tempo and onset strenght features
def compute_feats(audio, fs):
  onset_st = librosa.onset.onset_strength(audio,fs) # onset strenght method
  bpm = librosa.beat.tempo(audio, fs) # tempo method

  return onset_st, bpm


In [0]:
t_fts = np.zeros((audio_path_samples,2)) # array where new features will be saved
for index, f in enumerate(audio_path):
    audio, fs = librosa.load(f, sr=None)
    onset_st, bpm = compute_feats(audio, fs) # obtain new features from each audio file
    t_fts[index, 0] = np.mean(onset_st) # save mean of onset strenght feature into first column of array
    t_fts[index, 1] = bpm # save tempo into second column of array


In [0]:
new_feats = np.append(same_song, t_fts, axis=1) # add column to new features array to have song id's

np.savetxt('onset_and_tempo.csv', new_feats, fmt='%s', delimiter=',') # save computed features into csv file in order not to compute them again (takes a long time)


In [0]:
train_t_fts = t_fts[0:1395] # use first 1395 sample for training
test_t_fts = t_fts[1395:1744] # use remaining samples for testing

arousal_annts = annotations[' arousal_std'] # get [' ...'] column from annotations (output values)
train_ars = arousal_annts[0:1395] # training values
test_ars = arousal_annts[1395:1744] # testing values


Normalization

In [0]:
t_ft_max = np.max(train_t_fts, axis=0)
t_ft_min = np.min(train_t_fts, axis=0)
train_t_fts_norm = (train_t_fts - t_ft_min) / (t_ft_max - t_ft_min)
test_t_fts_norm = (test_t_fts - t_ft_min) / (t_ft_max - t_ft_min)

Regressor

In [206]:
reg_temp_ons = sklearn.linear_model.LinearRegression(normalize=True)
reg_temp_ons.fit(train_t_fts_norm, train_ars)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

Prediction and error values

In [208]:
reg_temp_ons_predict = reg_temp_ons.predict(test_t_fts_norm)

compute_metrics(test_ars,reg_temp_ons_predict)


Results : 
 MSE = 0.11983555708109095 
 R2 = -0.005558624649739885 



#Use given features to see if model behaves differently

Training set

In [0]:
train_files = [i + ".csv" for i in id_train_arr] # list with file name in features folder of train set

path_train_files = [os.path.join('/content/features',p) for p in train_files] # list with file path of train set


In [0]:
feats_params = ['pcm_RMSenergy_sma_amean','pcm_zcr_sma_amean','pcm_fftMag_spectralRollOff25.0_sma_amean','pcm_fftMag_spectralRollOff50.0_sma_amean',
                'pcm_fftMag_spectralRollOff75.0_sma_amean','pcm_fftMag_spectralRollOff90.0_sma_amean','pcm_fftMag_spectralFlux_sma_amean',
                'pcm_fftMag_spectralCentroid_sma_amean','pcm_fftMag_mfcc_sma[1]_amean','pcm_fftMag_mfcc_sma[2]_amean','pcm_fftMag_mfcc_sma[3]_amean',
                'pcm_fftMag_mfcc_sma[4]_amean','pcm_fftMag_mfcc_sma[5]_amean','pcm_fftMag_mfcc_sma[6]_amean','pcm_fftMag_mfcc_sma[7]_amean',
                'pcm_fftMag_mfcc_sma[8]_amean','pcm_fftMag_mfcc_sma[9]_amean','pcm_fftMag_mfcc_sma[10]_amean','pcm_fftMag_mfcc_sma[11]_amean',
                'pcm_fftMag_mfcc_sma[12]_amean','pcm_fftMag_mfcc_sma[13]_amean','pcm_fftMag_mfcc_sma[14]_amean']


############################## train inputs ####################################

feats_train_stats = []
for f in path_train_files:
  ufl_train_feats = pd.read_csv(f, sep=";", usecols=feats_params)
  feats_train_stats.append(ufl_train_feats.mean()) # has all the mean values for each parameter in each train file


In [0]:
train_feats = np.array(feats_train_stats) # numpy array of features statistics, each row is one song, each column a feature

############################### train outputs ##################################
arousal_mean_all = annotations[[' arousal_mean']] # get annotations of song id and arousal mean values only
arousal_std_all = annotations[[' arousal_std']] # get annotations of song id and arousal std values only
valence_mean_all = annotations[[' valence_mean']] # get annotations of song id and valence mean values only
valence_std_all = annotations[[' valence_std']] # get annotations of song id and valence std values only

train_index = annotations.index[annotations['song_id'].isin(id_train_arr)].tolist() # get indeces of the song ids which are equal to the train set

arousal_mean_train = []
arousal_std_train = []
valence_mean_train = []
valence_std_train = []
for i in train_index:
  arousal_mean_train.append(arousal_mean_all.iloc[i][-1].astype(float)) # mean arousal values for all id_train songs
  arousal_std_train.append(arousal_std_all.iloc[i][-1].astype(float)) # std arousal values for all id_train songs
  valence_mean_train.append(valence_mean_all.iloc[i][-1].astype(float)) # mean valence values for all id_train songs
  valence_std_train.append(valence_std_all.iloc[i][-1].astype(float)) # std valence values for all id_train songs


Test set

In [0]:
test_files = [i + ".csv" for i in id_test_arr] # list with file name in features folder of test set

path_test_files = [os.path.join('/content/features',p) for p in test_files] # list with file path of test set


In [0]:
############################## test inputs #####################################
feats_test_stats = []
for f in path_test_files:
  ufl_test_feats = pd.read_csv(f, sep=";", usecols=feats_params)
  feats_test_stats.append(ufl_test_feats.mean()) # has all the mean values for each parameter in each test file

In [0]:
test_feats = np.array(feats_test_stats) # numpy array of features statistics, each row is one song, each column a feature

############################### test outputs ###################################

test_index = annotations.index[annotations['song_id'].isin(id_test_arr)].tolist() # get indeces of the song ids which are equal to the train set
#arousal_train = [arousal_all.loc[i] for i in id_train]

arousal_mean_test = []
arousal_std_test = []
valence_mean_test = []
valence_std_test = []
for i in test_index:
  arousal_mean_test.append(arousal_mean_all.iloc[i][-1].astype(float)) # mean arousal values for all id_train songs
  arousal_std_test.append(arousal_std_all.iloc[i][-1].astype(float)) # std arousal values for all id_train songs
  valence_mean_test.append(valence_mean_all.iloc[i][-1].astype(float)) # mean valence values for all id_train songs
  valence_std_test.append(valence_std_all.iloc[i][-1].astype(float)) # std valence values for all id_train songs


Normalization

In [0]:
ft_max = np.max(train_feats, axis=0)
ft_min = np.min(train_feats, axis=0)
train_feats_norm = (train_feats - ft_min) / (ft_max - ft_min)
test_feats_norm = (test_feats - ft_min) / (ft_max - ft_min)

Regressors

In [218]:
lreg_amean = sklearn.linear_model.LinearRegression(normalize=True)
lreg_astd = sklearn.linear_model.LinearRegression(normalize=True)

lreg_vmean = sklearn.linear_model.LinearRegression(normalize=True)
lreg_vstd = sklearn.linear_model.LinearRegression(normalize=True)


lreg_amean.fit(train_feats_norm, arousal_mean_train)
lreg_astd.fit(train_feats_norm, arousal_std_train)

lreg_vmean.fit(train_feats_norm, valence_mean_train)
lreg_vstd.fit(train_feats_norm, valence_std_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [0]:
lreg_amean_predict = lreg_amean.predict(test_feats_norm)
lreg_amean_predict_l = lreg_amean_predict.tolist()

lreg_astd_predict = lreg_astd.predict(test_feats_norm)

lreg_vmean_predict = lreg_vmean.predict(test_feats_norm)

lreg_vstd_predict = lreg_vstd.predict(test_feats_norm)

In [0]:
def compute_metrics(y_tested,y_predicted):
  mse = sklearn.metrics.mean_squared_error(y_tested, y_predicted)
  r2 = sklearn.metrics.r2_score(y_tested, y_predicted)
  print("Results : \n MSE = {} \n R2 = {} \n".format(mse, r2))


In [221]:
compute_metrics(arousal_mean_test,lreg_amean_predict)

compute_metrics(arousal_std_test,lreg_astd_predict)

compute_metrics(valence_mean_test,lreg_vmean_predict)

compute_metrics(valence_std_test,lreg_vstd_predict)

Results : 
 MSE = 0.9902794820886152 
 R2 = 0.38716058546924415 

Results : 
 MSE = 0.13459126348922243 
 R2 = -0.010933330224322857 

Results : 
 MSE = 0.7711804540935766 
 R2 = 0.4158791965344998 

Results : 
 MSE = 0.1251682767190717 
 R2 = 0.016216817889952773 

